In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom
from morfeus import read_xyz, XTB
import numpy as np

def mol_to_xyz(smiles_list, N):
    results = []
    
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        
        params = rdDistGeom.ETKDG()
        params.numThreads = 0
        AllChem.EmbedMultipleConfs(mol, numConfs=N, params=params)
        
        for conf_id in range(mol.GetNumConformers()):
            num_atoms = mol.GetNumAtoms()
            xyz_lines = [str(num_atoms)]
            xyz_lines.append(smiles)
            conf = mol.GetConformer(conf_id)
            for atom in mol.GetAtoms():
                symbol = atom.GetSymbol()
                pos = conf.GetAtomPosition(atom.GetIdx())
                xyz_lines.append(f"{symbol} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")
            
            results.append("\n".join(xyz_lines))

    return results

def get_molecules(smiles_list, N):
    xyz_contents = mol_to_xyz(smiles_list, N)
    avg_fukui_electrophilicity = {}
    avg_fukui_nucleophilicity = {}

    for smiles in smiles_list:
        fukui_electrophilicity = []
        fukui_nucleophilicity = []

        conformer_xyz_contents = xyz_contents[:N]
        xyz_contents = xyz_contents[N:]

        for xyz_content in conformer_xyz_contents:
            xyz_file = f"temp.xyz"
            with open(xyz_file, 'w') as file:
                file.write(xyz_content)
            
            elements, coordinates = read_xyz(xyz_file)
            xtb = XTB(elements, coordinates)
            
            fukui_electrophilicity.append(xtb.get_fukui("electrophilicity"))
            fukui_nucleophilicity.append(xtb.get_fukui("nucleophilicity"))

        avg_electrophilicity = {atom_idx: np.mean([conf[atom_idx] for conf in fukui_electrophilicity]) 
                                for atom_idx in fukui_electrophilicity[0].keys()}
        avg_nucleophilicity = {atom_idx: np.mean([conf[atom_idx] for conf in fukui_nucleophilicity]) 
                               for atom_idx in fukui_nucleophilicity[0].keys()}

        avg_fukui_electrophilicity[smiles] = avg_electrophilicity
        avg_fukui_nucleophilicity[smiles] = avg_nucleophilicity

    if nucl_elec == "E":
        for smiles, values in avg_fukui_nucleophilicity.items():
            max_value = max(values.values())
            print(f"{smiles}: {max_value}")
    else:
        for smiles, values in avg_fukui_electrophilicity.items():
            max_value = max(values.values())
            print(f"{smiles}: {max_value}")

user_input = input("Enter a list of SMILES strings separated by commas: ")
smiles_list_input = user_input.split(',')
smiles_list_input = [smiles.strip() for smiles in smiles_list_input]

nucl_elec = input("Should we be ranking according to electrophilicity(E) or nucleophilicity(N)?")

get_molecules(smiles_list_input, 10) # Adjust N as needed


Enter a list of SMILES strings separated by commas:  CNO, CNC, CN, Nc1ccccc1
Should we be ranking according to electrophilicity(E) or nucleophilicity(N)? N


CNO: 0.2230633401343008
CNC: 0.22796323876850777
CN: 0.2575414853755686
Nc1ccccc1: 0.10114783055577989
